# 01.2 Scattering Compute Speed

**NOT COMPLETED**

In this notebook, the speed to extract scattering coefficients is computed.

In [2]:
import sys
import random
import os
sys.path.append('../src')
import warnings
warnings.filterwarnings("ignore") 
import torch
from tqdm import tqdm
from kymatio.torch import Scattering2D
import time
import kymatio.scattering2d.backend as backend

###############################################################################
# Finally, we import the `Scattering2D` class that computes the scattering
# transform.

from kymatio import Scattering2D


# 3. Scattering Speed Test

In [ ]:
# From: https://github.com/kymatio/kymatio/blob/0.1.X/examples/2d/compute_speed.py
# Benchmark setup
# --------------------
J = 3
L = 8
times = 10
devices = ['cpu', 'gpu']
scattering = Scattering2D(J, shape=(M, N), L=L, backend='torch_skcuda')
data = np.concatenate(dataset['img'],axis=0)
data = torch.from_numpy(data)
x = data[0:batch_size]

In [71]:
%%time
#mlflow.set_experiment('compute_speed_scattering')
for device in devices:
    #with mlflow.start_run():
        fmt_str = '==> Testing Float32 with {} backend, on {}, forward'
        print(fmt_str.format('torch', device.upper()))

        if device == 'gpu':
            scattering.cuda()
            x = x.cuda()
        else:
            scattering.cpu()
            x = x.cpu()

        scattering.forward(x)

        if device == 'gpu':
            torch.cuda.synchronize()

        t_start = time.time()
        for _ in range(times):
            scattering.forward(x)

        if device == 'gpu':
            torch.cuda.synchronize()

        t_elapsed = time.time() - t_start

        fmt_str = 'Elapsed time: {:2f} [s / {:d} evals], avg: {:.2f} (s/batch)'
        print(fmt_str.format(t_elapsed, times, t_elapsed/times))
#         mlflow.log_param('M',M)
#         mlflow.log_param('N',N)
#         mlflow.log_param('Backend', device.upper())
#         mlflow.log_param('J', J)
#         mlflow.log_param('L', L)
#         mlflow.log_param('Batch Size', batch_size)
#         mlflow.log_param('Times', times)
#         mlflow.log_metric('Elapsed Time', t_elapsed)
#         mlflow.log_metric('Average Time', times)

    ###############################################################################
    # The resulting output should be something like
    #
    # .. code-block:: text
    #
    #   ==> Testing Float32 with torch backend, on CPU, forward
    #   Elapsed time: 624.910853 [s / 10 evals], avg: 62.49 (s/batch)
    #   ==> Testing Float32 with torch backend, on GPU, forward


==> Testing Float32 with torch backend, on CPU, forward
Elapsed time: 523.081820 [s / 10 evals], avg: 52.31 (s/batch)
==> Testing Float32 with torch backend, on GPU, forward
Elapsed time: 16.777041 [s / 10 evals], avg: 1.68 (s/batch)
CPU times: user 53min 2s, sys: 4min 47s, total: 57min 50s
Wall time: 9min 54s
